In [1]:
import tensorflow as tf 
import numpy as np 
import keras 
from keras import layers
from Models import ActorCritic
import gym 

In [2]:
gamma = 0.99
max_steps_per_episode = 10000
env = gym.make("CartPole-v0")
epsilon = 1e-7
learning_rate = 0.01
num_actions = env.action_space.n

c:\dev\python\lib\site-packages\gym\envs\registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
c:\dev\python\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\dev\python\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
actor = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[critic, actor])

In [4]:


optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
huber_loss = tf.keras.losses.Huber()

log_probs = []
rewards_history = []
critic_values = []
running_reward = 0 
episodes = 0


In [5]:
while True :
    state = env.reset()
    episode_reward = 0

    with tf.GradientTape() as tape : 
        for i in range(max_steps_per_episode):
            env.render()
            state = tf.convert_to_tensor([state])
            value , action_probs = model(state)

            action = np.random.choice(num_actions,p=np.squeeze(action_probs))
            log_probs.append(tf.math.log(action_probs[0,action]))
            critic_values.append(value[0,0])

            state,reward,done,_ = env.step(action)

            rewards_history.append(reward)
            episode_reward +=reward

            if done :
                break
        

        running_reward = 0.05 * episode_reward +(0.95)*running_reward
        returns = []
        sum_rewards = 0
        for reward in rewards_history[::-1] : 
            sum_rewards = reward + gamma * sum_rewards
            returns.insert(0,sum_rewards)

        # Normalize 
        returns = np.array(returns)
        returns = (returns - np.mean(returns))/(np.std(returns)+epsilon)
        returns = list(returns)

        actor_loss = []
        critic_loss = []

        for log_prob,critic_value,ret in zip(log_probs,critic_values,returns):

            diff = ret - value 
            actor_loss.append(-log_prob*diff)
            critic_loss.append(huber_loss(tf.expand_dims(critic_value,0),tf.expand_dims(ret,0)))
        
        loss = sum(actor_loss) + sum(critic_loss)

        gradients = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(gradients,model.trainable_variables))
        
        log_probs = []
        rewards_history = []
        critic_values = []

        episodes +=1

    print(f"Episode {episodes} rewards are {running_reward}")

c:\dev\python\lib\site-packages\gym\core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1 rewards are 0.5
Episode 2 rewards are 2.025
Episode 3 rewards are 4.77375
Episode 4 rewards are 5.6350625
Episode 5 rewards are 7.803309375
Episode 6 rewards are 8.41314390625
Episode 7 rewards are 9.692486710937498
Episode 8 rewards are 11.107862375390622
Episode 9 rewards are 13.052469256621091
Episode 10 rewards are 12.949845793790036
Episode 11 rewards are 13.352353504100535
Episode 12 rewards are 13.834735828895507
Episode 13 rewards are 13.792999037450732
Episode 14 rewards are 13.803349085578194
Episode 15 rewards are 13.863181631299284
Episode 16 rewards are 14.07002254973432
Episode 17 rewards are 14.016521422247603
Episode 18 rewards are 13.915695351135222
Episode 19 rewards are 15.21991058357846
Episode 20 rewards are 15.608915054399537
Episode 21 rewards are 16.97846930167956
Episode 22 rewards are 16.77954583659558
Episode 23 rewards are 16.7405685447658
Episode 24 rewards are 16.95354011752751
Episode 25 rewards are 16.605863111651132
Episode 26 rewards are 16.6